In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from acquire import wrangle_zillow
from prepare import remove_outliers, train_val_test, x_y_split

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import stats
from env import get_connection
import os

In [151]:
def get_zillow_data():
    
    '''
    This function is used to get zillow data from sql database.
    '''
    
    if os.path.isfile('zillow.csv'):
        
        return pd.read_csv('zillow.csv')
    
    else:
        
        url = get_connection('zillow')
        query = '''
                SELECT * FROM properties_2017
                LEFT JOIN airconditioningtype USING(airconditioningtypeid)
                LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
                LEFT JOIN buildingclasstype USING(buildingclasstypeid)
                LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
                LEFT JOIN predictions_2017 USING(parcelid)
                LEFT JOIN propertylandusetype USING(propertylandusetypeid)
                LEFT JOIN storytype USING(storytypeid)
                LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
                WHERE transactiondate LIKE '2017%%'
                AND latitude IS NOT NULL
                AND longitude IS NOT NULL;
                '''
        df = pd.read_sql(query, url)
        df.to_csv('zillow.csv')
        return df

In [152]:
df = get_zillow_data()

In [153]:
df.drop(columns=['Unnamed: 0', 'id.1'], inplace=True)

In [175]:
def drop_duplicates(df):
    
    df.drop(columns=['Unnamed: 0', 'id.1'], inplace=True)
    
    df = df.sort_values('transactiondate')
    df = df[df.duplicated(subset=['parcelid'], keep='last')==False]
    
    return df

In [155]:
df = drop_duplicates(df)

In [156]:
def missing_values(df):
    
    missing_df = pd.DataFrame(df.isna().sum(), columns=['num_rows_missing'])
    
    missing_df['pct_rows_missing'] = missing_df['num_rows_missing'] / len(df)
    
    return missing_df

In [157]:
missing_values(df)

,num_rows_missing,pct_rows_missing
typeconstructiontypeid,77159,0.997131
storytypeid,77331,0.999354
propertylandusetypeid,0,0.000000
parcelid,0,0.000000
heatingorsystemtypeid,27941,0.361083
...,...,...
logerror,0,0.000000
transactiondate,0,0.000000
propertylandusedesc,0,0.000000
storydesc,77331,0.999354


## Prepare

In [158]:
def sfh(df):
    
    sp = [261, 266, 263, 275, 264]
    df = df[df['propertylandusetypeid'].isin(sp)]
    return df

In [173]:
df = sfh(df)

In [174]:
df.shape

(71587, 34)

In [168]:
def handle_missing_values(df, prop_required_col, prop_required_row):
    
    drop_cols = round(prop_required_col * len(df))
    df.dropna(thresh=drop_cols, axis=1, inplace=True)

    drop_rows = round(prop_required_row * len(df.columns))
    df.dropna(thresh=drop_rows, axis=0, inplace=True)
    


    return df

In [169]:
handle_missing_values(df, .4, .4)

,propertylandusetypeid,parcelid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,logerror,transactiondate,propertylandusedesc
0,261.0,14297519,NaN,1727539,3.5,4.0,NaN,3.5,3100.0,3100.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,NaN,0.025595,2017-01-01,Single Family Residential
1,261.0,17052889,NaN,1387261,1.0,2.0,NaN,1.0,1465.0,1465.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,NaN,0.055619,2017-01-01,Single Family Residential
2,261.0,14186244,NaN,11677,2.0,3.0,NaN,2.0,1243.0,1243.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,NaN,0.005383,2017-01-01,Single Family Residential
3,261.0,12177905,2.0,2288172,3.0,4.0,8.0,3.0,2376.0,2376.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,Central,-0.103410,2017-01-01,Single Family Residential
4,266.0,10887214,2.0,1970746,3.0,3.0,8.0,3.0,1312.0,1312.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,Central,0.006940,2017-01-01,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,266.0,10833991,2.0,2864704,3.0,3.0,8.0,3.0,1741.0,1741.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,Central,-0.002245,2017-09-20,Condominium
77575,261.0,11000655,2.0,673515,2.0,2.0,6.0,2.0,1286.0,1286.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,Central,0.020615,2017-09-20,Single Family Residential
77577,261.0,12773139,2.0,1843709,1.0,3.0,4.0,1.0,1032.0,1032.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,Central,0.037129,2017-09-21,Single Family Residential
77576,261.0,17239384,NaN,2968375,2.0,4.0,NaN,2.0,1612.0,1612.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,NaN,0.013209,2017-09-21,Single Family Residential
